In [1]:
import scipy.ndimage as ndi
import numpy as np
from skimage import io
import os
import glob
import tifffile
from tqdm import tqdm

from py3dbp import Packer, Bin, Item

In [2]:
def create_mask(im,intensity_threshold,iteration):
    h,_,_ = ndi.center_of_mass(im)
    h = int(h)
    
    bottom = 0
    for i in range(max(h-(20+iteration),0),0,-1):
        if im[i,:,:].max() < intensity_threshold:
            bottom = i
            break
    
    top = im.shape[0]
    for i in range(min(h+(20+iteration),im.shape[0]),im.shape[0]):
        if im[i,:,:].max() < intensity_threshold:
            top = i
            break

    mask = np.zeros(im.shape)
    mask[bottom:top,:,:] = 1
    
    im_avg = ndi.convolve(im, np.ones((3,3,3))/(3**3))
    
    if im[mask == 1].max() < im_avg.max():
        im[mask == 1] = 0
        return create_mask(im,intensity_threshold,iteration+1)
    return mask

In [14]:
file_paths = glob.glob("../../bugnist_512_oskar_filtered/**/*.tif")

In [4]:
picked_files = np.random.choice(file_paths,size=30,replace=False)
file = picked_files[0]

im = io.imread(file)
mask = create_mask(im, 100, 0)
im[mask == 0] = 0
# tifffile.imwrite("test-images/"+file.split("/")[-1], im)

In [16]:
picked_files = np.random.choice(file_paths,size=2,replace=False)

for file in tqdm(picked_files):
    im = io.imread(file)
    mask = create_mask(im, 100, 0)
    im[mask == 0] = 0
    # tifffile.imwrite("test-images/"+file.split("/")[-1], im)
    
    angle1 = np.random.randint(0,360)
    angle2 = np.random.randint(0,360)
    angle3 = np.random.randint(0,360)
    
    out = ndi.rotate(im, angle1, axes = (1,0))
    out = ndi.rotate(out, angle2, axes = (1,2))
    out = ndi.rotate(out, angle3, axes = (0,2))
    
    for i in range(out.shape[2]):
        if np.any(out[:,:,i] >= 100):
            ax2_top = i
    for i in range(out.shape[2]-1,-1,-1):
        if np.any(out[:,:,i] >= 100):
            ax2_bottom = i
    for i in range(out.shape[1]):
        if np.any(out[:,i,:] >= 100):
            ax1_top = i
    for i in range(out.shape[1]-1,-1,-1):
        if np.any(out[:,i,:] >= 100):
            ax1_bottom = i
    for i in range(out.shape[0]):
        if np.any(out[i,:,:] >= 100):
            ax0_top = i
    for i in range(out.shape[0]-1,-1,-1):
        if np.any(out[i,:,:] >= 100):
            ax0_bottom = i
    
    cut = out[ax0_bottom:ax0_top,ax1_bottom:ax1_top,ax2_bottom:ax2_top]

    tifffile.imwrite("test-images/smaller_cut_"+file.split("/")[-1], cut)

100%|█████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:08<00:00, 34.00s/it]


In [127]:
packer = Packer()
# desired_size = (900,650,650)
packer.add_bin(Bin('mix', 900, 650, 650, 999))

In [128]:
img_paths = glob.glob("test-images/*.tif")

In [129]:
img_dict = {}
for file in img_paths:
    im = io.imread(file)
    img_dict[file.split("/")[1][12:]] = im
    packer.add_item(Item(file.split("/")[-1][12:], im.shape[0], im.shape[1], im.shape[2], 1))

In [131]:
packer.pack(bigger_first=True)

In [132]:
bin = next(iter(packer.bins))

In [133]:
new_mix = np.zeros((900,650,650))

In [134]:
for item in bin.items:
    temp_img = img_dict[item.name]
    if item.rotation_type == 1:
        temp_img = np.swapaxes(temp_img,0,1)
    elif item.rotation_type == 2:
        temp_img = np.rollaxis(temp_img,0,3)
    elif item.rotation_type == 3:
        temp_img = np.swapaxes(temp_img,0,2)
    elif item.rotation_type == 4:
        temp_img = np.rollaxis(temp_img,2,0)
    elif item.rotation_type == 5:
        temp_img = np.swapaxes(temp_img,1,2)
    
    temp_img_shape = temp_img.shape
    start_position = item.position
    
    new_mix[int(start_position[0]):int(start_position[0])+temp_img_shape[0],
            int(start_position[1]):int(start_position[1])+temp_img_shape[1],
            int(start_position[2]):int(start_position[2])+temp_img_shape[2]] = temp_img
new_mix = new_mix.astype('uint8')
tifffile.imwrite("mixes/bigger_first_mix.tif", new_mix)